In [1]:
!pip install imutils

In [2]:
!pip install opencv-python

In [24]:
import cv2
import urllib
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import random , os , glob
from imutils import paths
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from urllib.request import urlopen

#Warningleri kapatmak için kullanılmaktadır.

import warnings
warnings.filterwarnings('ignore')

#Model değerlendirme için kullanılacak olan kütüphaneler
from sklearn.metrics import confusion_matrix, classification_report

# Model için kullanılacak olan kütüphaneler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, MaxPool2D, Dense, Dropout, SpatialDropout2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
dir_path = r"C:\Users\talha\Desktop\GarbageClassification\GarbageClassification\Garbage classification"

In [5]:
# Target_Size ve Label Değerlerinin Belirlenmesi

target_size = (224, 224)

waste_labels = {"cardboard": 0,
                "glass": 1,
                "metal" : 2,
                "paper" : 3,
                "plastic" : 4,
                "trash" : 5}

In [26]:
def load_datasets(path):
    """
Görsellerin bulunduğu dizindeki görüntüyü okuyup etiketlerini oluşturur.
Parametreler:
path: Görsellerin bulunduğu dizini ifade eder.
Return:
x: Görüntülere ait matris bilgilerini tutar.
labels: Görüntünün ait olduğu sınıf bilgisini tutan liste.
    """


    x = []
    labels = []

    # Gönderdiğimiz pathdeki görüntüleri listeleyip sıralamaktadır.
    image_paths = sorted(list(paths.list_images(path)))

    for image_path in image_paths:
        # Belirtilen pathdeki görüntüler openCV kütüphanesi ile okunmaktadır.
        img = cv2.imread(image_path)

        # Okunan görüntüler başlangıçta belirlenen target_size'a göre yeniden ölçeklendirilir.
        img = cv2.resize(img, target_size)

        # Ölçeklendirilen görüntüler x listesine eklenir.
        x.append(img)

        # Her bir path "/" ifadesi ile ayrıldığında dönen listenin sonran ikinci elamanı label'ı temsil etmektedir.
        label = image_path.split(os.path.sep)[-2]

        # Yakalanan labelların sayısal değer karşılıklarının olduğu waste_labels sözlüğü içerisinden gönderilen key
        # değerine karşılık value değeri alınarak label oluşturulur.
        labels.append(waste_labels[label])

    # Veri seti random bir şekilde karıştırılır.
    x, labels = shuffle(x, labels, random_state=42)

    # Boyut ve Sınıf bilgisi raporlanmaktadır.
    print(f"X boyutu: {np.array(x).shape}")
    print(f"Label sınıf sayısı: {len(np.unique(labels))} Gözlem sayısı: {len(labels)}")

    return x, labels

In [27]:
x, labes = load_datasets(dir_path)

X boyutu: (0,)
Label sınıf sayısı: 0 Gözlem sayısı: 0
